In [5]:
import torch

a = torch.arange(1, 25).float().reshape(2,3,4)
b = torch.rand(2, 3, 2)
c = b[..., 0, None]
a * c, c.shape, b, c

(tensor([[[ 0.7084,  1.4169,  2.1253,  2.8338],
          [ 3.3275,  3.9930,  4.6585,  5.3240],
          [ 1.8039,  2.0043,  2.2048,  2.4052]],
 
         [[ 0.8081,  0.8702,  0.9324,  0.9946],
          [10.3458, 10.9544, 11.5630, 12.1715],
          [ 6.3996,  6.7043,  7.0090,  7.3138]]]),
 torch.Size([2, 3, 1]),
 tensor([[[0.7084, 0.2739],
          [0.6655, 0.5484],
          [0.2004, 0.8385]],
 
         [[0.0622, 0.3439],
          [0.6086, 0.8304],
          [0.3047, 0.9468]]]),
 tensor([[[0.7084],
          [0.6655],
          [0.2004]],
 
         [[0.0622],
          [0.6086],
          [0.3047]]]))